---

## Test Your MAP_KEY

---

In [ ]:
MAP_KEY = '5e8bad8d50fa1ca84ea72175e2bace34'# change this key to user key

# check how many transactions we have
import pandas as pd
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)


transaction_limit             5000
current_transactions             0
transaction_interval    10 minutes
dtype: object

In [2]:
# let's create a simple function that tells us how many transactions we have used.
# We will use this in later examples

def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

Our current transaction count is 0


---

## API/data_availability

---

This service is designed to inform users about date range availability of our supported datasets.

For more information visit https://firms.modaps.eosdis.nasa.gov/api/data_availability

Let's see the full list of available sensors and their supported date ranges.


In [3]:
# let's query data_availability to find out what date range is available for various datasets
# we will explain these datasets a bit later

# this url will return information about all supported sensors and their corresponding datasets
# instead of 'all' you can specify individual sensor, ex:LANDSAT_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2025-01-01,2025-04-26
1,MODIS_SP,2000-11-01,2024-12-31
2,VIIRS_NOAA20_NRT,2025-02-01,2025-04-26
3,VIIRS_NOAA20_SP,2018-04-01,2025-01-31
4,VIIRS_NOAA21_NRT,2024-01-17,2025-04-26
5,VIIRS_SNPP_NRT,2025-02-01,2025-04-26
6,VIIRS_SNPP_SP,2012-01-20,2025-01-31
7,LANDSAT_NRT,2022-06-20,2025-04-26
8,GOES_NRT,2022-08-09,2025-04-26
9,BA_MODIS,2000-11-01,2024-12-01


**data_id** column shows the dataset id which we will need in later queries:
- 'NRT' means this is Near Real-Time dataset but it may also includes Real Time (RT) and Ultra Real Time (URT) data [click here more info on URT/RT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- 'SP' or Standard Processing; standard data products are an internally consistent, well-calibrated record of the Earth’s geophysical properties to support science. There is a multi-month lag in this dataset availability. [more information on SP vs NRT](https://www.earthdata.nasa.gov/data/tools/firms/faq)
- BA_MODIS is for MODIS burned areas product

**min_date** and **max_date** columns provide the available date range for these datasets. Dates are based on GMT

In [4]:
# now let's see how many transactions we use by querying this end point

start_count = get_transaction_count()
pd.read_csv(da_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

# now remember, after 10 minutes this will reset

We used 5 transactions.


---

## API/area

---

Fire detection hotspots based on area, date and sensor. For more information visit https://firms.modaps.eosdis.nasa.gov/api/area

The end point expects these parameters: [MAP_KEY], [SOURCE], [AREA_COORDINATES],[DAY_RANGE] and optionally [DATE] for historical data

**NOTE** - querying the entire world for VIIRS can return between 30,000 - 100,000+ records per day


In [5]:
# in this example let's look at VIIRS NOAA-20, entire world and the most recent day
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/world/1'
start_count = get_transaction_count()
df_area = pd.read_csv(area_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

df_area

We used 36 transactions.


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,66.81354,134.29771,331.17,0.51,0.66,2025-04-26,35,N20,VIIRS,n,2.0NRT,279.39,3.18,D
1,66.82079,134.29105,356.38,0.51,0.66,2025-04-26,35,N20,VIIRS,l,2.0NRT,283.30,8.61,D
2,66.82410,134.27792,342.71,0.51,0.66,2025-04-26,35,N20,VIIRS,n,2.0NRT,278.89,8.61,D
3,67.10844,135.13361,327.12,0.48,0.64,2025-04-26,35,N20,VIIRS,n,2.0NRT,279.97,2.18,D
4,67.15563,135.26959,340.43,0.47,0.64,2025-04-26,35,N20,VIIRS,n,2.0NRT,283.23,7.22,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25053,37.18055,-8.41155,334.30,0.42,0.45,2025-04-26,1357,N20,VIIRS,n,2.0NRT,301.98,2.67,D
25054,39.05598,-7.66544,327.13,0.48,0.48,2025-04-26,1357,N20,VIIRS,n,2.0NRT,297.43,2.51,D
25055,39.99049,-4.30304,340.72,0.37,0.58,2025-04-26,1357,N20,VIIRS,n,2.0NRT,299.92,3.51,D
25056,40.24209,-3.47183,336.61,0.41,0.61,2025-04-26,1357,N20,VIIRS,n,2.0NRT,297.93,3.96,D


In [6]:
# We can also focus on a smaller area ex. South Asia and get the last 3 days of records
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/54,5.5,102,40/3'
df_area = pd.read_csv(area_url)
df_area

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,6.84721,101.23814,367.00,0.35,0.56,2025-04-24,559,N20,VIIRS,h,2.0NRT,290.66,5.97,D
1,10.40709,98.63654,330.90,0.45,0.63,2025-04-24,601,N20,VIIRS,n,2.0NRT,283.05,1.40,D
2,10.51202,98.78631,329.03,0.44,0.62,2025-04-24,601,N20,VIIRS,n,2.0NRT,275.98,21.97,D
3,10.51282,98.79061,356.36,0.44,0.62,2025-04-24,601,N20,VIIRS,l,2.0NRT,276.43,38.87,D
4,10.51433,98.78635,349.86,0.44,0.62,2025-04-24,601,N20,VIIRS,n,2.0NRT,276.01,28.81,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34172,37.38245,55.11226,333.45,0.55,0.43,2025-04-26,854,N20,VIIRS,l,2.0NRT,308.43,2.13,D
34173,39.03753,54.25248,342.81,0.56,0.43,2025-04-26,854,N20,VIIRS,l,2.0NRT,313.68,10.57,D
34174,39.04137,54.25119,349.96,0.56,0.43,2025-04-26,854,N20,VIIRS,n,2.0NRT,316.41,13.80,D
34175,39.67253,64.77404,336.88,0.49,0.40,2025-04-26,854,N20,VIIRS,l,2.0NRT,308.58,11.11,D


List of supported countries and their 3-letter codes. This may be easier to read in html format https://firms.modaps.eosdis.nasa.gov/api/countries/?format=html however, you won't be able to see the exent box defined for each country.

Example below shows how you can view from Python.

In [7]:
# We can also focus on smaller area ex. South Asia and get last 3 days of records
countries_url = 'https://firms.modaps.eosdis.nasa.gov/api/countries'
df_countries = pd.read_csv(countries_url, sep=';')
df_countries

,id,abreviation,name,extent
0,1,ABW,Aruba,"BOX(-70.0624080069999 12.417669989,-69.8768204..."
1,2,AFG,Afghanistan,"BOX(60.4867777910001 29.3866053260001,74.89230..."
2,3,AGO,Angola,"BOX(11.6693941430001 -18.0314047239998,24.0617..."
3,4,AIA,Anguilla,"BOX(-63.4288223949999 18.1690941430001,-62.972..."
4,6,ALA,Aland Islands,"BOX(19.5131942070001 59.9044863950001,21.09669..."
...,...,...,...,...
239,234,WSM,Samoa,"BOX(-172.782582161 -14.052829685,-171.43769283..."
240,235,YEM,Yemen,"BOX(42.5457462900001 12.1114436720001,54.54029..."
241,236,ZAF,South Africa,"BOX(16.4699813160001 -46.965752863,37.97779381..."
242,237,ZMB,Zambia,"BOX(21.9798775630001 -18.0692318719999,33.6742..."


---

## API/country

---

Provides data specific to a country, although not recommended for large countries such as USA, China, Canada, Russia due to the complexity and size of their polygon shape which may cause the query to time out. To figure out the country code, see [/api/countries](https://firms.modaps.eosdis.nasa.gov/api/countries/?format=html) (example above).

In [8]:
# Let's see last four days MODIS data for Thailand
thai_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/THA/5'
df_thai = pd.read_csv(thai_url)
df_thai

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,THA,10.49355,99.17863,321.06,1.22,1.10,2025-04-22,744,Aqua,MODIS,54,6.1NRT,302.84,11.35,D
1,THA,13.99651,101.36413,325.22,1.00,1.00,2025-04-22,744,Aqua,MODIS,72,6.1NRT,302.88,10.32,D
2,THA,14.12012,100.50677,322.99,1.01,1.00,2025-04-22,744,Aqua,MODIS,69,6.1NRT,296.96,10.32,D
3,THA,14.27817,99.90459,324.64,1.04,1.02,2025-04-22,744,Aqua,MODIS,74,6.1NRT,301.84,9.15,D
4,THA,14.27913,100.46578,318.46,1.01,1.01,2025-04-22,744,Aqua,MODIS,47,6.1NRT,298.39,4.49,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,THA,9.98348,99.06538,315.40,1.02,1.01,2025-04-26,300,Terra,MODIS,41,6.1NRT,296.62,6.07,D
438,THA,11.04891,99.48295,315.81,1.03,1.02,2025-04-26,300,Terra,MODIS,49,6.1NRT,294.77,8.58,D
439,THA,15.29338,104.36448,327.05,2.32,1.47,2025-04-26,703,Aqua,MODIS,65,6.1NRT,298.39,40.27,D
440,THA,15.29503,104.36926,327.17,2.32,1.47,2025-04-26,703,Aqua,MODIS,66,6.1NRT,298.15,41.04,D


## Change data format "acq_time"

In [9]:
import datetime

# Ensure "acq_time" is a string, then pad with zeros
df_thai["acq_time"] = df_thai["acq_time"].astype(str).str.zfill(4)

# Convert to HH:MM format
df_thai["acq_time"] = pd.to_datetime(df_thai["acq_time"], format="%H%M").dt.time
# df_thai["acq_time"] = df_thai["acq_time"].apply(lambda x: datetime.datetime.strptime(x, "%H:%M:%S").time())

# Check the result
print(df_thai.dtypes)
print(df_thai["acq_time"].head())  # Display first few converted times

country_id     object
latitude      float64
longitude     float64
brightness    float64
scan          float64
track         float64
acq_date       object
acq_time       object
satellite      object
instrument     object
confidence      int64
version        object
bright_t31    float64
frp           float64
daynight       object
dtype: object
0    07:44:00
1    07:44:00
2    07:44:00
3    07:44:00
4    07:44:00
Name: acq_time, dtype: object


In [10]:
unique_values = df_thai["acq_time"].unique()
print(unique_values)

[datetime.time(7, 44) datetime.time(7, 46) datetime.time(14, 25)
 datetime.time(14, 27) datetime.time(19, 55) datetime.time(2, 36)
 datetime.time(8, 24) datetime.time(15, 5) datetime.time(15, 7)
 datetime.time(3, 15) datetime.time(3, 17) datetime.time(7, 24)
 datetime.time(14, 8) datetime.time(2, 17) datetime.time(2, 19)
 datetime.time(8, 1) datetime.time(8, 3) datetime.time(14, 44)
 datetime.time(14, 46) datetime.time(20, 14) datetime.time(2, 56)
 datetime.time(2, 58) datetime.time(3, 0) datetime.time(7, 3)
 datetime.time(7, 5)]


In [11]:
df_thai.head(100)

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,THA,10.49355,99.17863,321.06,1.22,1.10,2025-04-22,07:44:00,Aqua,MODIS,54,6.1NRT,302.84,11.35,D
1,THA,13.99651,101.36413,325.22,1.00,1.00,2025-04-22,07:44:00,Aqua,MODIS,72,6.1NRT,302.88,10.32,D
2,THA,14.12012,100.50677,322.99,1.01,1.00,2025-04-22,07:44:00,Aqua,MODIS,69,6.1NRT,296.96,10.32,D
3,THA,14.27817,99.90459,324.64,1.04,1.02,2025-04-22,07:44:00,Aqua,MODIS,74,6.1NRT,301.84,9.15,D
4,THA,14.27913,100.46578,318.46,1.01,1.01,2025-04-22,07:44:00,Aqua,MODIS,47,6.1NRT,298.39,4.49,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,THA,18.72292,98.18442,321.31,1.09,1.04,2025-04-22,07:44:00,Aqua,MODIS,59,6.1NRT,304.72,12.95,D
96,THA,18.74274,98.08300,312.31,1.10,1.05,2025-04-22,07:44:00,Aqua,MODIS,42,6.1NRT,293.67,8.97,D
97,THA,18.75041,98.24104,328.77,1.09,1.04,2025-04-22,07:44:00,Aqua,MODIS,74,6.1NRT,302.47,19.63,D
98,THA,18.75284,98.02817,348.39,1.11,1.05,2025-04-22,07:44:00,Aqua,MODIS,94,6.1NRT,300.30,57.79,D


## Hive partition

In [12]:
import numpy as np
import pandas as pd
import pyarrow

In [13]:
# Extract components from the "acq_date" and "acq_time" columns
df_thai["acq_year"] = pd.to_datetime(df_thai["acq_date"]).dt.year
df_thai["acq_month"] = pd.to_datetime(df_thai["acq_date"]).dt.month
df_thai["acq_day"] = pd.to_datetime(df_thai["acq_date"]).dt.day
df_thai["acq_hour"] = df_thai["acq_time"].apply(lambda x: x.hour)
df_thai["acq_minute"] = df_thai["acq_time"].apply(lambda x: x.minute)

# Write DataFrame to a directory "output_parquet" partitioned by retrieval_time
df_thai.to_parquet(
    "df_thai",
    partition_cols=["acq_year", "acq_month", "acq_day", "acq_hour", "acq_minute"],
    engine="pyarrow",
    index=False
)